<a href="https://colab.research.google.com/github/21cs076/Real-Time-Landslide-Detection-and-Buried-Vehicle-Identification-using-YOLOv8/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install roboflow

from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov10n.pt')

# Perform inference on an image from Google Drive
results = model('https://ultralytics.com/images/bus.jpg')

# Iterate through the results and display each
for result in results:
    result.show()

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="4MptKsSsONT0fnQsIDc2")
project = rf.workspace("deslizamentos").project("landslide-detection-augmentation")
version = project.version(12)
dataset = version.download("yolov8")


In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
model = YOLO("yolov10n.pt")

# Train the model on your Landslide dataset
model.train(
    data="/content/Landslide-Detection-Augmentation-12/data.yaml",  # Path to your data.yaml file
    epochs=100,  # Number of epochs (adjust based on your needs)
    batch=16,   # Batch size (adjust based on GPU memory), (16), 32, 64, 128
    imgsz=736,  # Image size for training ((640 is a good choice), 736, 832), 896 pixels.
    project="runs/train",  # Directory to save training results
    name="landslide_detection",  # Name for the training run
    exist_ok=True  # Overwrite existing results if any
)

model.save("/content/Landslide-Detection-Augmentation-12/best736.pt")

In [5]:
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("/content/Landslide-Detection-Augmentation-12/best736.pt")
# Evaluate the model using the val() method
metrics = model.val(data="/content/Landslide-Detection-Augmentation-12/data.yaml", task="detect")

# Print evaluation metrics
print(f"mAP: {metrics.box.maps}")
print(f"mAP50: {metrics.box.map50}")
print(f"mAP75: {metrics.box.map75}")  # mAP at IoU thresholds of 50% (mAP50) and 75% (mAP75)

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10n summary (fused): 285 layers, 2,695,196 parameters, 0 gradients, 8.2 GFLOPs


val: Scanning /content/Landslide-Detection-Augmentation-12/valid/labels.cache... 115 images, 37 backgrounds, 0 corrupt: 100%|██████████| 115/115 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:05<00:00,  1.34it/s]


                   all        115        162       0.66      0.431      0.478       0.22
               Fissure         29         60       0.61      0.333       0.39      0.167
             Landslide         61        102       0.71      0.529      0.566      0.273
Speed: 0.4ms preprocess, 11.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val
mAP: [    0.16666     0.27272]
mAP50: 0.47777222619063353
mAP75: 0.2192795409876977


In [ ]:
from google.colab import files
uploaded = files.upload()

import os
os.listdir('/content')

In [ ]:
!pip install onnx

import torch
import onnx
import numpy as np

# Assuming 'model' is already defined and loaded with your trained model
onnx_model_path = "/content/Landslide-Detection-Augmentation-12/lsmodel.onnx"
torch.onnx.export(
    model,
    torch.randn(1, 3, 640, 640),  # Dummy input for the model
    onnx_model_path,
    opset_version=11,
    input_names=["input"],
    output_names=["output"]
)

print(f"Model has been converted to ONNX format and saved at: {onnx_model_path}")

In [ ]:
# Clone YOLOv8 repository
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics